# INFOSYS_POPULATION Data Wrangling
 - Replace missing value with 'N' for
   - Scored_Rec_Decision          
   - Scored_Flag                   
   - Elan_Flag                     
   - CLI_or_New_Prod_Flag          
   - Financial_Template_Found_Flag

 -  Create derived variables 
    - Yer : Year derived from 'Revw_Month'
    - Qty : Quarter derived from 'Revw_Quarter'
    - Mth : Month derived from 'Revw_Month'
    - ElRmFl_ind: Indicator if 'Existing_Limit' + 'Request_Amount' = 'Final_Limit'
    - card_overlay: Indicator if  (One_Card > 0 ) & ( Corporate_Card > 0  or Purchasing_Card > 0))
    - cmb_flg: Combination of 'Decision' and 'Scored_Rec_Decision'


In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import matplotlib.pyplot as plt

from pandas.api.types import is_numeric_dtype

from Features import *
from AttributeRelevance import *

In [2]:
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)  

In [3]:
vc = lambda df, feature: df[feature].value_counts(dropna = False)/len(df)
vu = lambda df, fea: len(df[fea].unique())

In [4]:
def cr_dfa (df, df_var_dsc):
    """
    Purpose:
      Create 3  dataframe  
        - da1: get data type
        - da2: get null value count
        - da3: get ratio of null value count
      create dfa  by concating da1, da2, da3 with axis = 1 (column)
    Parameter
     input 
      df : df to be analyzed
      df_var_dsc: df ffor variable description
     output 
      dfa : dataframe for analyzed result    
    """
    
    # get data types$
    da1 = df.dtypes.to_frame(name = 'dtype')
     # get null value count
    da2 = df.isnull().sum().to_frame(name = 'missing value')

    # get ratio of null value count
    sa3 = round(df.isnull().sum()/len(df) * 100, 3)
    da3 = sa3.to_frame(name = 'Percent of missing value')
    #da3 = da3.rename(columns={'index':'Variable'}, inplace = True)
    dfa = pd.concat([da1, da2, da3, df_var_dsc], axis = 1)
    return dfa

In [5]:
def unq_val(df, l_col):
    """
    Purpose:
      - Create dataframe to include  
        - df0: keep feature name 
        - df1: get count for unique values from the feature
        - df2: get ratio of count for unique values from the feature 
        - df3: Merge df1 and df2 
      - Concating dfy, df0, df3 with axis = 0 
    Parameter
     input 
      df : df to be analyzed 
      l_col:  list of features
     output 
      dfy : dataframe for the count of unique values and  ratio of unique values    
    """
    dfy =  pd.DataFrame() 
    l_comment=[]
    for fea in l_col:   
       
        if vu(df, fea) < 15:  
           df0 = pd.DataFrame([[fea,'','', ' ']],columns=['feature','value','cnt','ratio_cnt'] )
           df1 = df[fea].value_counts().to_frame(name = 'cnt').reset_index()
           df1.rename(columns={'index':'value'}, inplace = True)
           sd =  df[fea].value_counts()/len(df)
           df2 = sd.to_frame(name='ratio_cnt').reset_index()
           df2.rename(columns={'index':'value'}, inplace = True)
           df3 = df1.merge(df2, on = 'value',how = 'inner')
           dfy = pd.concat([dfy,  df0, df3], axis = 0)


    return dfy  
   

## Read variable description from xlsx file

In [6]:
path = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\'
df_lup  = pd.read_excel(path + "usb_variable_description.xlsx", sheet_name = 'Variable description', engine = 'openpyxl' )
l_key = df_lup['Variable'].tolist()
l_var = df_lup['Description'].tolist()
d_var_dsc = dict(zip(l_key, l_var))               
d_var_dsc['NAICS_Cd']
df_lup.set_index('Variable', inplace = True) # Set Variable as index
df_lup
#d_var_dsc

,Description
Variable,
Org_Struc_Desc,Organization Structure Description
NAICS_Cd,The North American Industry Classification Sys...
NAICS_Desc,"NAICS code description. e.g., Roofing Contractors"
Revw_Id,Unique ID of the Credit Application
Revw_Month,Calendar month in which application was submit...
Revw_Quarter,Calendar quarter in which application was subm...
Decision,Final disposition on the application
Scored_Flag,"'Y' if decision was automated, otherwise empt..."
Elan_Flag,"'Y' if cases sourced by indirect channel, oth..."


## Read data from xlsx file

In [7]:
path = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\'
df = pd.read_excel(path + "INFOSYS_POPULATION.xlsx", sheet_name = 'INFOSYS_POPULATION', engine = 'openpyxl' )
df.head(3).T

,0,1,2
Org_Struc_Desc,Corporation,Partnership,Non-Profit
NAICS_Cd,238160,621210,813319
NAICS_Desc,Roofing Contractors,Offices of Dentists,Other Social Advocacy Organizations
Revw_Id,117882,118031,119338
Revw_Month,2019/01,2019/01,2019/02
Revw_Quarter,2019/1,2019/1,2019/1
Decision,Approved,Declined,Approved
Scored_Flag,NaN,NaN,NaN
Elan_Flag,NaN,NaN,NaN
CLI_or_New_Prod_Flag,Y,Y,NaN


## Get  df attribute before data wrangling

In [8]:
dfa_b = cr_dfa (df, df_lup)
dfa_b.head()

,dtype,missing value,Percent of missing value,Description
Org_Struc_Desc,object,0,0.0000,Organization Structure Description
NAICS_Cd,int64,0,0.0000,The North American Industry Classification Sys...
NAICS_Desc,object,0,0.0000,"NAICS code description. e.g., Roofing Contractors"
Revw_Id,int64,0,0.0000,Unique ID of the Credit Application
Revw_Month,object,0,0.0000,Calendar month in which application was submit...


## Get count of unique values before data wrangling
  - Invoke unq_val to crearte a dataframe to keep to keep
    - count unique value
    - Ratio of count unique value

In [9]:
df_val_b = unq_val(df, df.columns)


## Data Wrangling

In [10]:
df['Scored_Rec_Decision'].replace(np.NaN,  'Declined', inplace  = True) 
df['Scored_Flag'].replace(np.NaN, 'N',  inplace  = True)
df['Elan_Flag'].replace(np.NaN,'N', inplace  = True)
df['CLI_or_New_Prod_Flag'].replace(np.NaN,'N', inplace  = True )
df['Financial_Template_Found_Flag'].replace(np.NaN,  'N', inplace  = True)

# Create new fields 
#df['Yer']= list( map(lambda str : int(str.split('/')[0]), df['Revw_Month']))
#df['Qty']= list( map(lambda str : int(str.split('/')[1]), df['Revw_Quarter']))
#df['Mth']= list( map(lambda str : int(str.split('/')[1]), df['Revw_Month']))
#df['ElRmFl_ind']  = (df['Existing_Limit'] + df['Request_Amount']) == df['Final_Limit']
#df['card_overlay'] = (df.One_Card > 0 ) & (( df.Corporate_Card > 0 ) | (df.Purchasing_Card > 0))
#df['cmb_flg']= list( map(lambda str1, str2 : str1[0:1] + str2[0:1], df['Decision'],df['Scored_Rec_Decision']))


## Rearrange column name

In [11]:
#new_idx = [ 'Revw_Month','Yer', 'Mth', 'Revw_Quarter','Qty', 
#           'Org_Struc_Desc', 'NAICS_Cd', 'NAICS_Desc', 'Revw_Id',
#        'Scored_Flag', 'Elan_Flag',
#       'CLI_or_New_Prod_Flag', 'Channel', 'Corporate_Card', 'Purchasing_Card',
#       'One_Card', 'card_overlay', 'Other_NonBankCard', 'SBFE_Score',
#       'CC_Score', 'Financial_Template_Found_Flag', 'Existing_Limit',
#       'Request_Amount', 'Final_Limit','ElRmFl_ind', 'Decision',  'Scored_Rec_Decision', 'cmb_flg', ]
#df = df.reindex(columns = new_idx)
#df.head(3).T

## Get  df attribute after data wrangling

In [12]:
dfa_a = dfa (df)
dfa_a

NameError: name 'dfa' is not defined

## Get count of unique values after data warangling 
 - Invoke unq_val to crearte a dataframe to keep to keep     
    - count unique value 
    - Ratio of count unique value 

In [14]:
l_col = df.columns
#l_col_eli = ['Mth', 'Qty']
#l_col = [ elm for elm in l_col if elm not in l_col_eli]
df_val_a = unq_val(df, l_col)

## Save the dataframes into excel

In [15]:
path_file = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\INFOSYS_POPULATION_yue_22sep22.xlsx'
writer = pd.ExcelWriter(path_file)

df.to_excel(writer, sheet_name = 'INFOSYS_POPULATION_new', index = False )
dfa_b.to_excel(writer, sheet_name = 'vbr attri before data wrangle', index = True )
#df_val_b.to_excel(writer, sheet_name = 'vbr unq val before data wrangle', index = False )
#dfa_a.to_excel(writer, sheet_name = 'vbr attri after data wrangle', index = True )
df_val_a.to_excel(writer, sheet_name = 'vbr unq val after data wrangle', index = False )
writer.save()

In [ ]:
stop

In [ ]:
path_file = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\INFOSYS_POPULATION_yue.csv'

df.to_csv(path_file, index = False )

In [ ]:
feats_dict = {}

for col in [c for c in df.columns if c != 'Decision']:
    if is_numeric_dtype(df[col]):
        feats_dict[col] = ContinuousFeature(df, col)
    else:
        feats_dict[col] = CategoricalFeature(df, col)

feats = list(feats_dict.values())

In [ ]:
ar = AttributeRelevance()

In [ ]:
iv = IV()
ar.bulk_iv(feats, iv).head()

In [ ]:
df.groupby('Decision').agg({'Org_Struc_Desc': 'count'})

In [ ]:
df.groupby('Org_Struc_Desc').agg({'Decision': 'count'})

In [ ]:
df.groupby('Decision').agg('count')

In [ ]:
df.groupby('Yer').agg('sum')

In [ ]:
df.groupby('Yer').sum()

In [ ]:
df.groupby('Yer')['Org_Struc_Desc'].sum()

In [ ]:
df.groupby(['Yer', 'Decision'])['Scored_Flag'].count()  # not include NaN

In [ ]:
df.groupby(['Yer', 'Decision'])['Scored_Flag'].size()  # include NaN

In [ ]:
2019  0           523
      1           176
2020  0           405
      1           164
2021  0           567
      1           185
2022  0           355
      1           108
    
	
349	523
162	176
267	405
149	164
369	567
160	185
201	355
88	108
1745	2483
    

In [ ]:
vu = lambda df, fea: len(df[fea].unique())
vu(df, 'Financial_Template_Found_Flag')

In [ ]:
vc(df, 'Financial_Template_Found_Flag')

In [ ]:
df['cmb_flg']= list( map(lambda str1, str2, str3 : str1[0:1] + str1[0:1] , 
                    df['Decision'],df['Scored_Rec_Decision']]))
            

In [ ]:
df['cmb_flg'].head()

In [ ]:
df['ElRmFl_ind']  = (df['Existing_Limit'] + df['Request_Amount']) == df['Final_Limit']


In [ ]:
#df['card_overlay + df.Purchasing_Card
# #df['card_overlay + df.Purchasing_Card
#df[(df.One_Card > 0 ) & (( df.Corporate_Card > 0 ) | (df.Purchasing_Card > 0))].head()  # DF
#(df.One_Card > 0 ) & (( df.Corporate_Card > 0 ) | (df.Purchasing_Card > 0))  #  False | True
df['card_overlay'] = (df.One_Card > 0 ) & (( df.Corporate_Card > 0 ) | (df.Purchasing_Card > 0))

# nw df[[(df['One_Ca#df['card_overlay + df.Purchasing_Cardrd'] > 0 ) & ( df['Corporate_Card'] > 0 | df['Purchasing_Card'] > 0)]] 
# (df['One_Card'] > 0 ) & ( df['Corporate_Card'] > 0 | df['Purchasing_Card'] > 0) 
#( (df['Corporate_Card'] > 0) | (df['Purchasing_Card'] > 0)) # work

In [ ]:
s1 = vc(df, 'card_overlay')
s1.head()


In [ ]:
l_col = ['Org_Struc_Desc','card_overlay']
fea = 'card_overlay'
df1 = df[fea].value_counts().to_frame(name = 'cnt')
df2 = df[fea].value_counts()/len(df).to_frame(name='pct') 
#df3 = df1.merge(df2, on = 'Org_Struc_Desc',,how = 'inner )
#df3.head()

In [ ]:
df0 = pd.DataFrame([[fea,'','']],columns=['value','cnt','pct_cnt'] )
df0

In [ ]:
df_val_a = unq_val(df, l_col)
df_val_a 

In [ ]:
dfy =  pd.DataFrame()
df['cmb_flg']= list( map(lambda str1, str2 : str1[0:1] + str2[0:1], df['Decision'],df['Scored_Rec_Decision']))

for fea in l_col:  
    #print(f' fea: {fea} vu(df, fea) {vu(df, fea)}')
    if vu(df, fea) < 15:  
       df0 = pd.DataFrame([[fea,'','']],columns=['value','cnt','pct_cnt'] )
       df1 = df[fea].value_counts().to_frame(name = 'cnt').reset_index()
       df1.rename(columns={'index':'value'}, inplace = True)
       sd =  df[fea].value_counts()/len(df)
       df2 = sd.to_frame(name='pct_cnt').reset_index()
       df2.rename(columns={'index':'value'}, inplace = True)
       df3 = df1.merge(df2, on = 'value',how = 'inner')
       dfy = pd.concat([dfy,  df0, df3], axis = 0)
dfy
    

In [ ]:
dfy =  pd.DataFrame()
dfy = pd.concat([dfy, df0, df3], axis = 1)
dfy

In [ ]:
dfy =  pd.DataFrame()
dfy = pd.concat([dfy, df0, df3], axis = 0)
dfy

In [ ]:
df3

In [ ]:
pd.concat([df0, df3], axis = 0)

In [ ]:
def dfa (df):
    # get data types$
    da1 = df.dtypes.to_frame(name = 'dtype')
     # get null value count
    da2 = df.isnull().sum().to_frame(name = 'mval')

    # get ratio of null value count
    sa3 = df.isnull().sum()/len(df)
    da3 = sa3.to_frame(name = 'rto_mval')
    #da3 = da3.rename(columns={'index':'Variable'}, inplace = True)
    dfa = pd.concat([da1, da2, da3], axis = 1)
    return dfa

In [ ]:
dfa = dfa (df)
dfa

In [ ]:
da1 = df.dtypes.to_frame(name = 'dtype').reset_index()
#da1.rename(columns={'index':'Variable'}, inplace = True) 

da2 = df.isnull().sum().to_frame(name = 'mis').reset_index()
#da2.rename(columns={'index':'Variable'}, inplace = True)

#type(df.dtypes)

sa3 = df.isnull().sum()/len(df)
da3 = sa3.to_frame(name = 'pct_mis').reset_index()
#da3 = da3.rename(columns={'index':'Variable'}, inplace = True)
da4 = pd.concat([da1, da2, da3], axis = 1)
da4

In [ ]:
#df2 = pd.DataFrame([vc(df, feature) for feature in df.columns], columns=['Variable', 'count'])
df2 = df.isnull().sum().to_frame(name = 'mis').reset_index()
da2.rename(columns={'index':'Variable'}, inplace = True)
df2

In [ ]:
sf3 = df.isnull().sum()/len(df)
df3 = sf3.to_frame(name = 'pct_mis').reset_index()
df3.rename(columns={'index':'Variable'}, inplace = True)
df3

In [ ]:
#[vc(df, feature) for feature in df.columns]
s = df.isnull().sum()

In [ ]:
df.count()

In [ ]:
d_var_dsc

In [ ]:
dfa_b


In [ ]:
path_file = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\test.xlsx'
writer = pd.ExcelWriter(path_file)

dfa_b.to_excel(writer, sheet_name = 'variables', index = True )

writer.save()

In [ ]:
df_val_a 

In [ ]:
dfy =  pd.DataFrame()   
l_col =['Org_Struc_Desc']
for fea in l_col:        
    if vu(df, fea) < 15:  
       df0 = pd.DataFrame([[fea,'','', ' ']],columns=['feature','value','cnt','ratio_cnt'] )
       df1 = df[fea].value_counts().to_frame(name = 'cnt').reset_index()
       df1.rename(columns={'index':'value'}, inplace = True)
       sd =  df[fea].value_counts()/len(df)
       df2 = sd.to_frame(name='ratio_cnt').reset_index()
       df2.rename(columns={'index':'value'}, inplace = True)
       df3 = df1.merge(df2, on = 'value',how = 'inner')
       dfy = pd.concat([dfy,  df0, df3], axis = 0)

In [ ]:
comment= ''
l_comment=[]
fea = 'Org_Struc_Desc'
for index, row in df3.iterrows():
    #if row['value'] != '':
    comment = comment + row['value'] + f" ({row['ratio_cnt']:.2%})" +', '    
       #comment = comment + f'row['ratio_cnt'] + ', '
l_comment.append([fea, comment[0:-2]]) 
l_comment

In [ ]:
pd.DataFrame(l_comment, columns = ['fea', 'note']) 

In [ ]:
df3

In [ ]:
path_file = 'C:\\Users\\sophia.yue\\Infosys\\Sandeep Project\\usb_val.xlsx'
writer = pd.ExcelWriter(path_file)

df_val_a.to_excel(writer, sheet_name = 'variables', index = True )
writer.save()

In [ ]:
dfa_b 

In [ ]:
dfy =  pd.DataFrame() 
l_comment=[]
for fea in df.columns:   
    print(f'fea {fea}')
    if vu(df, fea) < 15:  
       df0 = pd.DataFrame([[fea,'','', ' ']],columns=['feature','value','cnt','ratio_cnt'] )
       df1 = df[fea].value_counts().to_frame(name = 'cnt').reset_index()
       df1.rename(columns={'index':'value'}, inplace = True)
       sd =  df[fea].value_counts()/len(df)
       df2 = sd.to_frame(name='ratio_cnt').reset_index()
       df2.rename(columns={'index':'value'}, inplace = True)
       df3 = df1.merge(df2, on = 'value',how = 'inner')
       dfy = pd.concat([dfy,  df0, df3], axis = 0)

       comment = ''
       for index, row in df3.iterrows():
           comment = comment + row['value'] + f" ({row['ratio_cnt']:.2%})" +', '   
           
       l_comment.append([fea, comment[0:-2]]) 

In [ ]:
df3

In [ ]:
comment = ''
for index, row in df3.iterrows():
    print(f" row {row['value']}")
    if str(row['value']).isalpha(): 
       comment = comment + row['value'] + f" ({row['ratio_cnt']:.2%})" +', '
    else:
       #comment = comment + f"{row['value']:1}" + f" ({row['ratio_cnt']:.2%})" +', '
       comment = comment + str(int(row['value'])) + f" ({row['ratio_cnt']:.2%})" +', ' 
       
comment

In [ ]:
comment

In [ ]:
df1, df2  = unq_val(df, df.columns)

In [19]:
def unq_valx(df, l_col):
    """
    Purpose:
      - Create dataframe to include  
        - df0: keep feature name 
        - df1: get count for unique values from the feature
        - df2: get ratio of count for unique values from the feature 
        - df3: Merge df1 and df2 
      - Concating dfy, df0, df3 with axis = 0 
    Parameter
     input 
      df : df to be analyzed 
      l_col:  list of features
     output 
      dfy : dataframe for the count of unique values and  ratio of unique values    
    """
    dfy =  pd.DataFrame() 
    l_comment=[]
    for fea in l_col:   
       
        if vu(df, fea) < 15:  
           df0 = pd.DataFrame([[fea,'','', ' ']],columns=['feature','value','cnt','ratio_cnt'] )
           df1 = df[fea].value_counts().to_frame(name = 'cnt').reset_index()
           df1.rename(columns={'index':'value'}, inplace = True)
           sd =  df[fea].value_counts()/len(df)
           df2 = sd.to_frame(name='ratio_cnt').reset_index()
           df2.rename(columns={'index':'value'}, inplace = True)
           df3 = df1.merge(df2, on = 'value',how = 'inner')
           dfy = pd.concat([dfy,  df0, df3], axis = 0)

           comment = ''
           for index, row in df3.iterrows():
               print(f" row {row['value']}")
               if str(row['value']).isdigit(): 
                  comment = comment + str(int(row['value'])) + f" ({row['ratio_cnt']:.2%})" +', '    
                  
               else:
                  comment = comment + row['value'] + f" ({row['ratio_cnt']:.2%})" +', '  
           l_comment.append([fea, comment[0:-2]]) 
           print (f' {fea}, {comment[0:-2]}')
   
    dfx = pd.DataFrame(l_comment, columns = ['fea', 'note'])  
    dfx = dfx.set_index('fea', inplace = True)
    return dfx, dfy  
   

In [21]:
l_col = ['Corporate_Card', 'Purchasing_Card',
       'One_Card', 'Other_NonBankCard', 'Scored_Rec_Decision', 'SBFE_Score',
       'CC_Score', 'Financial_Template_Found_Flag', 'Existing_Limit',
       'Request_Amount', 'Final_Limit']
      
dfx1, dfy1 = unq_valx(df, l_col)

 row Corporation
 row LLC
 row Non-Profit
 row Partnership
 row Other
 row Sole Proprietorship
 row MSA Av Consumer
 Org_Struc_Desc, Corporation (50.58%), LLC (24.97%), Non-Profit (20.90%), Partnership (2.86%), Other (0.56%), Sole Proprietorship (0.08%), MSA Av Consumer (0.04%)
 row 2022/2
 row 2021/2
 row 2021/3
 row 2021/4
 row 2019/2
 row 2019/1
 row 2022/1
 row 2019/3
 row 2021/1
 row 2020/4
 row 2020/1
 row 2020/3
 row 2019/4
 row 2020/2
 Revw_Quarter, 2022/2 (11.56%), 2021/2 (8.34%), 2021/3 (7.97%), 2021/4 (7.85%), 2019/2 (7.85%), 2019/1 (7.57%), 2022/1 (7.09%), 2019/3 (6.89%), 2021/1 (6.12%), 2020/4 (5.96%), 2020/1 (5.92%), 2020/3 (5.92%), 2019/4 (5.84%), 2020/2 (5.11%)
 row Approved
 row Declined
 Decision, Approved (74.51%), Declined (25.49%)
 row N
 row Y
 Scored_Flag, N (70.28%), Y (29.72%)
 row Y
 row N
 Elan_Flag, Y (62.99%), N (37.01%)
 row N
 row Y
 CLI_or_New_Prod_Flag, N (75.19%), Y (24.81%)
 row Indirect
 row Commercial Banking
 row CPS Only
 row Community Banking
 ro

TypeError: can only concatenate str (not "numpy.float64") to str

In [22]:
df.columns

Index(['Org_Struc_Desc', 'NAICS_Cd', 'NAICS_Desc', 'Revw_Id', 'Revw_Month',
       'Revw_Quarter', 'Decision', 'Scored_Flag', 'Elan_Flag',
       'CLI_or_New_Prod_Flag', 'Channel', 'Corporate_Card', 'Purchasing_Card',
       'One_Card', 'Other_NonBankCard', 'Scored_Rec_Decision', 'SBFE_Score',
       'CC_Score', 'Financial_Template_Found_Flag', 'Existing_Limit',
       'Request_Amount', 'Final_Limit'],
      dtype='object')

In [23]:
l_col = ['Corporate_Card', 'Purchasing_Card',
       'One_Card', 'Other_NonBankCard', 'Scored_Rec_Decision', 'SBFE_Score',
       'CC_Score', 'Financial_Template_Found_Flag', 'Existing_Limit',
       'Request_Amount', 'Final_Limit']
      
dfx1, dfy1 = unq_valx(df, l_col)

 row 0.0


TypeError: can only concatenate str (not "numpy.float64") to str